In [1]:
import pandas as pd
import numpy as np

import hdbscan

from pathlib import Path

In [2]:
# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_d2v_umap5.npy')  # 0.766 0.027 0.948
# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_d2v_ppa2_umap5.npy')  # 0.729 0.017 0.948
# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_d2v_v2_umap5.npy')  # 0.747 0.026 0.948
# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_d2v_v2_ppa2_umap5.npy')  # 0.692 0.019 0.948
# path_embedding = Path('../../data/emb/emb_tutby_126784_bert_umap5.npy')  # 0.699 0.017 0.948
# path_embedding = Path('../../data/emb/emb_tutby_126784_bert_ppa2_umap5.npy')  # 0.680 0.027 0.948
path_embedding = Path('../../data/emb/emb_clean_tutby_126784_w2v_idf_umap5.npy')  # 0.792 0.032 0.948
# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_w2v_idf_ppa2_umap5.npy')  # 0.766 0.030 0.948

path_corpus = Path('../../data/corpora/tutby_126784.csv')

In [3]:
data = pd.read_csv(path_corpus)

data['tags'] = data['tags'].apply(eval)
data['tags'] = data['tags'].apply(lambda tags: [tag.lower().strip() for tag in tags])

print(data.shape)
display(data.head(5))

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,Главный тренер солигорского «Шахтера» Олег Куб...,[футбол]
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,Планы по восстановлению рисунка есть. Но пока ...,"[архитектура, живопись, жкх]"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,Областное управление МЧС ко Дню пожарной служб...,[министерства]
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,Чемпион Беларуси БАТЭ воспользовался паузой в ...,"[футбол, батэ]"
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,Генпрокуратура Украины откроет уголовное произ...,"[ситуация в украине, государственные перевороты]"


In [4]:
with open(path_embedding, 'rb') as file:
    embeddings = np.load(file)
    
index = ~np.isnan(embeddings).any(axis=1)
data = data[index]
embeddings = embeddings[index]
    
print(embeddings.shape)

(126773, 5)


In [5]:
%%time

cluster = hdbscan.HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom')  # cluster_selection_epsilon
cluster.fit(embeddings)

labels = cluster.labels_

print(len(labels))
print(sum(labels == -1) / len(labels))

126773
0.41922964669133017
Wall time: 8.78 s


In [6]:
%%time

index = np.arange(len(data))
index_pairs = np.random.choice(index, size=(100000000, 2))

pair_labels = labels[index_pairs]
pair_tags = data['tags'].values[index_pairs]

same_labels = (pair_labels[:, 0] == pair_labels[:, 1]) & (pair_labels[:, 0] != -1) & (pair_labels[:, 1] != -1)
same_tags = np.array([len(set(tag0) & set(tag1)) > 0 for tag0, tag1 in pair_tags])

precision = (same_labels & same_tags).sum() / same_labels.sum()
recall = (same_labels & same_tags).sum() / same_tags.sum()
rand_index = (same_labels == same_tags).mean()

print(pair_labels.shape)
print(pair_tags.shape)

print(precision)
print(recall)
print(rand_index)

(100000000, 2)
(100000000, 2)
0.7916268101186699
0.031691701513996244
0.94848254
Wall time: 4min 29s
